Caso Práctico
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/IPython-for-data-science/blob/master/05-caso-practico/05-caso-practico.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/IPython-for-data-science/blob/master/05-caso-practico/05-caso-practico.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/IPython-for-data-science/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/IPython-for-data-science/blob/master/readme.md)

Este directorio contiene la información de cuatro (4) estaciones automáticas de medición de viento. Se desear realizar un proceso de transformación de los datos con el fin de producir la información requerida para construir un atlas eólico de la región. A continuación se presentan los requerimientos solicitados y su respectiva solución.

**Requerimiento 1.--** Generar una tabla unificada que contenga las siguientes columnas:

* Nombre de la estación.
* Fecha en el formato YYYY-MM-DD.
* Año.
* Mes.
* Dia.
* HHMMSS.
* Hora (HH).
* Minuto (MM).
* Dirección.
* Velocidad.

In [1]:
import glob
import re

In [9]:
##
## archivo de salida
##
file_out = open('tabla1.csv', 'w')
  
## se agrega el encabezamiento
file_out.write("ESTACION,FECHA,ANO,MES,DIA,HHMMSS,HORA,MIN,DIR,VEL\n")
    
##
## se recorren todos los archivos *csv en el directorio 
##
for filename in glob.glob("Es*.csv"):
    
    ## abre el archivo para lectura
    file_in = open(filename, 'r')
    
    ## se ignora la primera linea de encabezados
    isfirstline = True
    
    ## se leen una a una las lineas del archivo *csv
    for line in file_in:
        
        ## se ignora la primera linea de encabezados
        if isfirstline is True:
            isfirstline = False
            continue
        
        ##
        ## se agrega el nombre del archivo
        ##
        line = filename[:-4] + ';' + line
        
        ##
        ## Se cambian las ',' por '.'
        ##
        line = line.replace(',', '.')
        
        ##
        ## Se cambian los ';' por ','
        ##
        line = line.replace(';', ',')
        
        ##
        ## Se extrae de HH:MM:SS las HH y los MM
        ##
        groups = re.search(',([0-9]+):([0-9][0-9]):([0-9][0-9])', line)
        if groups:
            org_text = groups.group(0)
            new_text = org_text + ',' + groups[1] + ',' + groups[2]
            line = line.replace(org_text, new_text)
        
        ##
        ## Se agrega el 0 al digito del dia
        ##
        groups = re.search(',([0-9])/', line)
        if groups:
            org_text = groups.group(0)
            new_text = '0' + groups.group(1) + '/'
            line = line.replace(org_text, new_text)


        ## Se cambia el formato DD/MM/YY a YYYY-MM-DD
        ## y agrega las columnas año, mes, dia
        groups = re.search('([0-9][0-9])\/([0-9][0-9])\/([0-9][0-9])', line)  
        org_text = groups.group(0)
        new_text = '20' + groups.group(3) + '-' + groups.group(2) + '-' + groups.group(1) 
        new_text +=  ',20' + groups.group(3) + ',' + groups.group(2) + ',' + groups.group(1) 
        
        # print(lineaorg_text, '\n', new_text, sep = '')
        line = line.replace(org_text, new_text)
        
        ##
        ## se escribe la linea modificada al archivo de salida
        ##
        file_out.write(line)
    
    file_in.close()

file_out.close()

In [10]:
!head tabla1.csv 

ESTACION,FECHA,ANO,MES,DIA,HHMMSS,HORA,MIN,DIR,VEL
Estacion1,2005-04-16,2005,04,16,11:10:00,11,10,135,6.3
Estacion1,2005-04-16,2005,04,16,11:20:00,11,20,135,5.1
Estacion1,2005-04-16,2005,04,16,11:30:00,11,30,135,6.3
Estacion1,2005-04-16,2005,04,16,11:40:00,11,40,113,6.1
Estacion1,2005-04-16,2005,04,16,11:50:00,11,50,135,4.1
Estacion1,2005-04-16,2005,04,16,12:00:00,12,00,135,5.5
Estacion1,2005-04-16,2005,04,16,12:10:00,12,10,135,5.4
Estacion1,2005-04-16,2005,04,16,12:20:00,12,20,135,5.5
Estacion1,2005-04-16,2005,04,16,12:30:00,12,30,90,4.6


**Requerimiento 2.--** Generar una tabla unificada que contenga las siguientes columnas:

* Nombre de la estación.
* Año.
* Mes.
* Dia.
* Hora.
* Velocidad media del viento para la correspodiente hora.

In [52]:
file_in = open('tabla1.csv', 'r')
isfirstline = True
dicvel = {}
for line in file_in:

    ## se ignora la primera linea de encabezados
    if isfirstline is True:
        isfirstline = False
        continue

    line = line.split(',')
    clave = line[0] + '-' + line[1] + '-' + line[6]
    
    if clave in dicvel.keys():
        dicvel[clave] = dicvel[clave] + [float(line[-1])] 
    else:
        dicvel[clave] = [float(line[-1])] 
    
file_in.close()

dicvel_prom = dicvel.copy()


for clave in dicvel:
    dicvel_prom[clave] = round(sum(dicvel[clave]) / len(dicvel[clave]), 2)
    
file_out = open('tabla2.csv', 'w')

## se agrega el encabezamiento
file_out.write("ESTACION,ANO,MES,DIA,HORA,VEL\n")

for clave in dicvel_prom:
    text =  clave + '-' + str(dicvel_prom[clave])
    text = text.replace('-', ',')
    file_out.write(text + '\n')
    
file_out.close()




In [45]:
dicvel


{'Estacion1-2005-04-16-11': [6.3, 5.1, 6.3, 6.1, 4.1],
 'Estacion1-2005-04-16-12': [5.5, 5.4, 5.5, 4.6, 6.7, 5.0],
 'Estacion1-2005-04-16-13': [5.5, 5.8, 5.3, 4.2, 3.8, 4.6],
 'Estacion1-2005-04-16-14': [4.5, 3.7, 2.8, 3.4, 4.6, 3.0],
 'Estacion1-2005-04-16-15': [2.5, 3.1, 2.6, 4.1, 4.2, 4.3],
 'Estacion1-2005-04-16-16': [3.3, 4.3, 4.3, 3.9, 2.3, 4.1],
 'Estacion1-2005-04-16-17': [4.6, 5.6, 5.1, 6.1, 4.3, 3.3],
 'Estacion1-2005-04-16-18': [4.1, 4.3, 4.1, 4.5, 5.4, 6.2],
 'Estacion1-2005-04-16-19': [4.3, 4.0, 2.5, 4.8]}

**Requerimiento 3.--** Generar una tabla que contenga las siguientes columnas:

* Estación.
* Año.
* Mes.
* Velocidad media del viento para ese mes y ese año.

In [ ]:
## para resolver en clase

**Requerimiento 4.--** Generar una tabla que contenga las siguientes columnas:

* Estación.
* Año.
* Velocidad media para dicho año.

In [ ]:
## para resolver en clase

**Requerimiento 5.--** Generar una tabla que contenga las siguientes columnas:

* Estación.
* Mes.
* Velocidad media para ese mes calculada sobre todos los datos.

In [ ]:
## para resolver en clase

**Requerimiento 6.--** Generar una tabla que contenga las siguientes columnas:

* Estación.
* Hora.
* Velocidad media para ese hora calculada sobre todos los datos.

In [ ]:
## para resolver en clase

**Requerimiento 7.--** Generar una tabla que contenga las siguientes columnas:

* Estación.
* Mes (MM).
* Hora (HH).
* Velocidad media para ese mes y esa hora calculada sobre todos los datos.

In [ ]:
## para resolver en clase

Caso Práctico
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/IPython-for-data-science/blob/master/05-caso-practico/05-caso-practico.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/IPython-for-data-science/blob/master/05-caso-practico/05-caso-practico.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/IPython-for-data-science/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/IPython-for-data-science/blob/master/readme.md)